In [34]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import openpyxl
import os

In [21]:

file_path = '/Users/robertstillwell/Downloads/2022-nsduh-detailed-tables/NSDUHDetTabsSect6pe2022.htm'
with open(file_path, 'r') as file:
    html = file.read()
soup = bs(html, 'html.parser')
tables = soup.find_all('table')
len(tables)


174

In [22]:
import xml.etree.ElementTree as ET

def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Extract data from the XML
    data = []
    for variable in root.findall(".//variable"):
        variable_data = {
            "code": variable.get("code"),
            "label": variable.get("label"),
            "type": variable.get("type"),
            "display_code": variable.get("display-code"),
            "use_finder": variable.get("use-finder"),
            "values": []
        }
        for value in variable.findall("value"):
            value_data = {
                "code": value.get("code"),
                "label": value.get("label")
            }
            variable_data["values"].append(value_data)
        data.append(variable_data)

    return data






data = parse_xml(file_path)


ParseError: not well-formed (invalid token): line 7, column 42 (<string>)

In [36]:

# for entry in data:
#     print(f"Code: {entry['code']}, Label: {entry['label']}, Type: {entry['type']}")
locations = ['Census Region', 'State', 'County', 'None']
demographics = ['Race','Gender','Hispanic Origin','Ten-Year Age Groups','Five-Year Age Groups','Single-Year Ages', 'None','None','None']
# for each location, select each permutation of 3 demographics
call = 0
selection_sets = []
ss_in_order = []
for location in locations:
    imax = len(demographics)-2
    for i in range(0,imax):
        jmax = len(demographics)-1
        for j in range(i+1,jmax):
            kmax = len(demographics)
            for k in range(j+1,kmax):
                choices = [location, demographics[i], demographics[j], demographics[k]]
                choices = [x for x in choices if x != 'None']
                selection_sets.append(choices) if choices not in selection_sets else None
                # if the location + demographics are not in order then skip this iteration

table = pd.DataFrame(selection_sets, columns=['Location','Demographic 1','Demographic 2','Demographic 3'])
table.reset_index(inplace=True, drop=False)
table.rename(columns={'index':'Call Number'}, inplace=True)

results_path = '/Users/robertstillwell/Mapping_Documents/NSDUH_Call_list.xlsx'
directory = os.path.dirname(results_path)
if not os.path.exists(directory):
    os.makedirs(directory)
if os.path.exists(results_path):
    os.remove(results_path)
table.to_excel(results_path, index=False)



In [28]:
table

,Call Number,Location,Demographic 1,Demographic 2,Demographic 3
0,0,Census Region,Race,Race,Race
1,1,Census Region,Race,Race,Gender
2,2,Census Region,Race,Race,Hispanic Origin
3,3,Census Region,Race,Race,Ten-Year Age Groups
4,4,Census Region,Race,Race,Five-Year Age Groups
...,...,...,...,...,...
611,611,None,None,None,None
612,612,None,None,None,None
613,613,None,None,None,None
614,614,None,None,None,None


In [4]:
import xml.etree.ElementTree as ET

def parse_icd10_and_race_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    icd10_data = {}
    race_data = {}

    # Extract ICD10 code data
    for variable in root.findall(".//variable[@code='D77.V2']"):
        for hier_level in variable.findall("hier-level"):
            level_code = hier_level.get("code")
            level_label = hier_level.get("label")
            icd10_data[level_code] = {
                "label": level_label,
                "counts_by_race": {}
            }

    # Extract race data and link with ICD10
    for variable in root.findall(".//variable[@code='D77.V8']"):
        for value in variable.findall("value"):
            race_code = value.get("code")
            race_label = value.get("label")
            race_data[race_code] = race_label

    # Example of how to integrate counts (mockup, adjust according to your XML structure)
    for icd_code in icd10_data:
        for race_code in race_data:
            # Here you would extract the actual counts from your XML structure
            icd10_data[icd_code]["counts_by_race"][race_code] = "mockup_count"

    return icd10_data, race_data

file_path = "/Users/robertstillwell/Downloads/Multiple Cause of Death, 1999-2020-resp.xml"
icd10_data, race_data = parse_icd10_and_race_data(file_path)
    
# Print the data
for icd_code, details in icd10_data.items():
    print(f"ICD10 Code: {icd_code}, Label: {details['label']}")
    for race_code, count in details["counts_by_race"].items():
        print(f"  Race: {race_data[race_code]}, Count: {count}")


ICD10 Code: D77.V2-level1, Label: UCD - ICD Chapter
  Race: American Indian or Alaska Native, Count: mockup_count
  Race: Asian or Pacific Islander, Count: mockup_count
  Race: Black or African American, Count: mockup_count
  Race: White, Count: mockup_count
ICD10 Code: D77.V2-level2, Label: UCD - ICD Sub-Chapter
  Race: American Indian or Alaska Native, Count: mockup_count
  Race: Asian or Pacific Islander, Count: mockup_count
  Race: Black or African American, Count: mockup_count
  Race: White, Count: mockup_count
ICD10 Code: D77.V2-level3, Label: Underlying Cause of death
  Race: American Indian or Alaska Native, Count: mockup_count
  Race: Asian or Pacific Islander, Count: mockup_count
  Race: Black or African American, Count: mockup_count
  Race: White, Count: mockup_count
